# What this notebook is for?

This notebook aids the process of evaluating accuracy measures and predicting class outputs of already trained models.

In [10]:
import tensorflow as tf
import numpy as np
import h5py as h5

## Reading data

Throughout this project we've used these functions in order to read the processed data.

In [11]:
def make_dimensions_compatible(arr):
    
    return arr.reshape(arr.shape[0],-1,1)

In [12]:
def load_dataset(dataset_iter=1, window_size=512):
    global X_train, Y_train, X_dev, Y_dev, X_test, Y_test
    
    dataset_relative_path = 'dataset/random-iter-%d/' % dataset_iter
    
    datafile = dataset_relative_path + 'datafile%d.h5' % window_size

    with h5.File(datafile, 'r') as datafile:
        X_train = np.array(datafile['X_train'])
        Y_train = np.array(datafile['Y_train'])

        X_dev = np.array(datafile['X_dev'])
        Y_dev = np.array(datafile['Y_dev'])

        X_test = np.array(datafile['X_test'])
        Y_test = np.array(datafile['Y_test'])
        
        # setting the rank of the data to be compatible with 1d convolution functions
        # defined in tensorflow
        X_train = make_dimensions_compatible(X_train)
        X_dev = make_dimensions_compatible(X_dev)
        X_test = make_dimensions_compatible(X_test)
        
        # normalization
        X_train = X_train / 1000
        X_dev = X_dev / 1000
        X_test = X_test / 1000

In [56]:
def get_session_path(dataset_iter=1, window_size=512, model_num=1, model_prefix='cnn', model_suffix='_lr-0.00002_mbs-128'):
    return ('train/dataset-%d-%d/' + model_prefix + '%d' + model_suffix + '/') % (window_size, dataset_iter, model_num)



## Prediction

In [57]:
def predict(X_test, session_path, model_file, Y_test_onehot=None):

    tf.reset_default_graph()

    checkpoint_path = session_path
    model_path = session_path + model_file

    with tf.Session() as sess:
        loader = tf.train.import_meta_graph(model_path)
        loader.restore(sess, tf.train.latest_checkpoint(checkpoint_path))

        graph = tf.get_default_graph()

        X = graph.get_tensor_by_name('X:0')
        Y = graph.get_tensor_by_name('Y:0')
        is_train = graph.get_tensor_by_name('is_train:0')
        
#         epoch_counter = graph.get_tensor_by_name('epoch_counter:0')
#         print(epoch_counter.eval())

        Y_hat = graph.get_tensor_by_name('softmax_output:0')

        predict_op = tf.argmax(Y_hat, 1)

        y_hat_test = predict_op.eval({X: X_test, is_train: False})
        
        # print the accuracy of the test set if the labels are provided
        if (Y_test_onehot is not None):
            y_test = np.argmax(Y_test_onehot, 1)
            acc = (y_hat_test == y_test).mean()
        

    return y_hat_test, acc


## Prediction with majority voting on ensemble network

In [58]:
def predict_voting(X_test_voting, session_path, model_file):

    tf.reset_default_graph()

    checkpoint_path = session_path
    model_path = session_path + model_file
    
    y_hat_test_voting = []

    with tf.Session() as sess:
        loader = tf.train.import_meta_graph(model_path)
        loader.restore(sess, tf.train.latest_checkpoint(checkpoint_path))

        graph = tf.get_default_graph()

        X = graph.get_tensor_by_name('X:0')
        is_train = graph.get_tensor_by_name('is_train:0')

        Y_hat = graph.get_tensor_by_name('softmax_output:0')

        predict_op = tf.argmax(Y_hat, 1)
        
        classname, idx, counts = tf.unique_with_counts(predict_op)
        predict_voting_op = tf.gather(classname, tf.argmax(counts))

        # no. of training examples with the original feature size
        m = X_test_voting.shape[0]
        
        # no. of split training examples of each original example
        m_each = X_test_voting.shape[1]
        
        for ex in range(m):
            x_test_voting = make_dimensions_compatible(X_test_voting[ex])
            pred = predict_voting_op.eval({X: x_test_voting, is_train: False})
            
            y_hat_test_voting.append(pred)

    return y_hat_test_voting

### Accuracy measure

In [98]:
model_num = 4
window_size = 1024
dataset_iters = (1, 2, 3, 4, 5, 6)

accuracies = np.array([])

for dataset_iter in dataset_iters:
    load_dataset(dataset_iter=dataset_iter, window_size=window_size)
    predictions, acc = predict(X_test, get_session_path(dataset_iter=dataset_iter, window_size=window_size, model_num=model_num), 'model.meta', Y_test_onehot=Y_test)
    accuracies = np.append(accuracies, acc)

accuracies = accuracies * 100
print(accuracies)

print("Mean accuracy: %f" % accuracies.mean())

INFO:tensorflow:Restoring parameters from train/dataset-1024-1/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-2/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-3/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-4/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-5/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-6/cnn4_lr-0.00002_mbs-128/model
[ 99.42857143 100.         100.          98.85714286  95.42857143
  98.28571429]
Mean accuracy: 98.666667


### Single model accuracy with voting measure

In [97]:
model_num = 4
window_size = 1024
dataset_iters = (1, 2, 3, 4, 5, 6)

accuracies_voting = np.array([])

for dataset_iter in dataset_iters:
    dataset_relative_path = 'dataset/random-iter-%d/' % dataset_iter
    testfile = (dataset_relative_path + 'testset_voting_%d.h5') % window_size
    session_path = get_session_path(dataset_iter=dataset_iter, window_size=window_size, model_num=model_num)
    model_file = 'model.meta'

    with h5.File(testfile, 'r') as testfile:
        X_test_voting = testfile['X']
        X_test_voting = np.array(X_test_voting) / 1000
        y_test_voting = np.array(testfile['Y'])

        y_hat_test_voting = predict_voting(X_test_voting, session_path, model_file)

        acc_voting_model = (y_test_voting == y_hat_test_voting).mean()
        accuracies_voting = np.append(accuracies_voting, acc_voting_model)


accuracies_voting = accuracies_voting * 100
print("Accuracy with voting: ", accuracies_voting)
print("Mean accuracy with voting: %f" % accuracies_voting.mean())
print("Standard deviation: %f" % accuracies_voting.std())


INFO:tensorflow:Restoring parameters from train/dataset-1024-1/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-2/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-3/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-4/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-5/cnn4_lr-0.00002_mbs-128/model
INFO:tensorflow:Restoring parameters from train/dataset-1024-6/cnn4_lr-0.00002_mbs-128/model
Accuracy with voting:  [100. 100. 100. 100.  98. 100.]
Mean accuracy with voting: 99.666667
Standard deviation: 0.745356
